In [17]:
import torch
import torch.nn as nn

net1 = nn.Linear(3, 7, bias=False)
net2 = nn.Conv2d(3, 7, 1, 1, bias=False)

params1 = net1.state_dict()['weight']
print(params1.shape)

params2 = net2.state_dict()['weight']
print(params2.shape)

# # data = torch.randn(2, 3, 4, 4).permute(0, 2, 3, 1)
data = torch.randn(2, 3, 4, 4)

# out = net2(data)
# print(out.shape)

# print('net1: \n', net1.weight.shape)
# print('net2: \n', net2.weight.shape)

data1, data2, data3 = data[:,0,...], data[:,1,...], data[:,2,...]
print(data1, data2)

torch.Size([7, 3])
torch.Size([7, 3, 1, 1])
tensor([[[ 0.6684, -0.4807, -0.4296,  0.3449],
         [ 0.3263,  0.7188, -0.0091, -0.4603],
         [-1.4662,  0.6485,  0.6784, -0.0166],
         [-0.3397, -0.0255,  1.0837,  0.5459]],

        [[-1.6927, -1.7381, -1.4663,  0.7579],
         [-0.6431,  0.1603, -0.3115,  3.2425],
         [ 0.1155,  1.2633,  0.0644,  0.2882],
         [ 0.7730, -0.4666,  0.6517,  0.4776]]]) tensor([[[ 1.8022, -0.3740,  0.3732,  0.0203],
         [-1.8460,  0.7974,  0.1499,  0.7085],
         [-0.1511, -0.8546,  0.2014, -0.1625],
         [ 0.8767,  0.6290,  1.8556, -0.3418]],

        [[-1.4471, -0.4469,  1.1689,  2.1712],
         [-0.3812,  0.2507,  1.0345, -1.3663],
         [-0.6040,  1.4030,  1.0606,  0.0100],
         [ 0.1147, -0.1028, -0.6135,  1.0051]]])


In [22]:
import torch

path_src = '/home/hjf/workspace/mmpretrain/work_dirs/pretrained_in1k/04swin/swin_tiny_224_b16x64_300e_imagenet_20210616_090925-66df6be6.pth'
path_out = '/home/hjf/workspace/mmpretrain/work_dirs/pretrained_in1k/08cluster/weights_from_swin-t.pth'

params = torch.load(path_src, map_location='cpu')
# print(params.keys())

if 'state_dict' in params.keys():
    params = params['state_dict']

params_new = {}
for k, v in params.items():
    if k.startswith('backbone.norm'):
        new_k = k.replace('backbone.norm', 'backbone.norm3')

    elif 'attn.w_msa.qkv.weight' in k:
        new_k = k.replace('attn.w_msa.qkv.weight', 'token_mixer.qkv.weight')
        v = v.unsqueeze(-1).unsqueeze(-1)
    elif 'attn.w_msa.qkv.bias' in k:
        new_k = k.replace('attn.w_msa.qkv.bias', 'token_mixer.qkv.bias')

    elif 'attn.w_msa.proj.weight' in k:
        new_k = k.replace('attn.w_msa.proj.weight', 'token_mixer.proj.weight')
        v = v.unsqueeze(-1).unsqueeze(-1)
    elif 'attn.w_msa.proj.bias' in k:
        new_k = k.replace('attn.w_msa.proj.bias', 'token_mixer.proj.bias')

    elif 'ffn.layers.0.0.weight' in k:
        new_k = k.replace('ffn.layers.0.0.weight', 'mlp.fc1.weight')
        v = v.unsqueeze(-1).unsqueeze(-1)
    elif 'ffn.layers.0.0.bias' in k:
        new_k = k.replace('ffn.layers.0.0.bias', 'mlp.fc1.bias')

    elif 'ffn.layers.1.weight' in k:
        new_k = k.replace('ffn.layers.1.weight', 'mlp.fc2.weight')
        v = v.unsqueeze(-1).unsqueeze(-1)
    elif 'ffn.layers.1.bias' in k:
        new_k = k.replace('ffn.layers.1.bias', 'mlp.fc2.bias')

    else:
        new_k = k
    params_new[new_k] = v
torch.save(params_new, path_out)
print('Done!')

Done!
